# Library


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
import os

In [3]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# Fold

In [4]:
df1_fold = 'D:/data/complete/big2/BD1/df'

# server

In [5]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSE')

In [6]:
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

# Load
- 등록정보&제원정보 -> result 4등급(연료 전체)
- 정기검사&정밀검사

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [7]:
# 8.6s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
}
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1906876 entries, 0 to 1906875
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   VIN                object 
 1   BSPL_STDG_CD       int64  
 2   EXHST_GAS_GRD_CD   object 
 3   EXHST_GAS_CERT_NO  object 
 4   VHCL_ERSR_YN       object 
 5   MANP_MNG_NO        object 
 6   YRIDNW             int64  
 7   VHCTY_CD           object 
 8   PURPS_CD2          object 
 9   FRST_REG_YMD       float64
 10  VHCL_FBCTN_YMD     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 160.0+ MB


In [8]:
carr = car.rename(columns=car_ch_col)
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [9]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566615 entries, 0 to 566614
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   MANP_MNG_NO   566615 non-null  object
 1   FUEL_CD       545937 non-null  object
 2   VHCTY_TY_CD2  553038 non-null  object
 3   MNFCTR_NM     559794 non-null  object
 4   VHCNM         566544 non-null  object
 5   VHCL_FRM      536460 non-null  object
 6   EGIN_TY       517013 non-null  object
dtypes: object(7)
memory usage: 30.3+ MB


In [10]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 정기검사(STD_TB_JGT_HIS)

In [11]:
# 3m 34.9s
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}
jgt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65903838 entries, 0 to 65903837
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   VIN                     object 
 1   FDRM_INSP_INSP_MTHD_CD  object 
 2   FDRM_INSP_KND_CD        object 
 3   FDRM_INSP_JGMT          object 
 4   FDRM_NLOD_SMO_MSTVL1    float64
 5   FDRM_NLOD_SMO_MSTVL2    float64
 6   FDRM_NLOD_SMO_MSTVL3    float64
 7   FDRM_NLOD_SMO_JT_YN1    object 
 8   FDRM_INSP_YMD           int64  
 9   FDRM_DRVNG_DSTNC        float64
 10  FDRM_NLOD_SMO_PRMVL1    float64
dtypes: float64(5), int64(1), object(5)
memory usage: 5.4+ GB


In [12]:
jgtr = jgt.rename(columns=jgt_ch_col)
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 정밀검사(STD_TB_EET_HIS_ME)

In [13]:
# 6m 36.1s
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1 FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}
eet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102191938 entries, 0 to 102191937
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VIN                    object 
 1   PRCINSP_MSRMT_MTHD_CD  object 
 2   PRCINSP_KND_CD         object 
 3   PRCINSP_JGMT           object 
 4   PREC_NLOD_SMO_MSTVL1   float64
 5   PREC_NLOD_SMO_MSTVL2   float64
 6   PREC_NLOD_SMO_MSTVL3   float64
 7   PREC_NLOD_SMO_JT_YN1   object 
 8   PRCINSP_YMD            int64  
 9   PRCINSP_DRVNG_DSTNC    float64
 10  PREC_NLOD_SMO_PRMVL1   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.4+ GB


In [14]:
eetr = eet.rename(columns=eet_ch_col)
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [15]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       47916 non-null  int64 
 1   STDG_CTPV_NM  47916 non-null  object
 2   STDG_SGG_NM   47892 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [16]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## RH에서 제공한 법정동코드

In [17]:
# 1.3s
rh = we.export_to_pandas("SELECT DONG_CODE, CTPRVN_NM, SIGNGU_NM FROM STD_BD_TB_MAPDATA;")
rh = rh.rename(columns={'DONG_CODE':'법정동코드', 'CTPRVN_NM':'시도', 'SIGNGU_NM':'시군구'})
rh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동코드   3799 non-null   int64 
 1   시도      3799 non-null   object
 2   시군구     3757 non-null   object
dtypes: int64(1), object(2)
memory usage: 89.2+ KB


## 4등급 result(for DPF유무)

In [18]:
# 3.3s
rs = we.export_to_pandas("SELECT 차대번호, DPF유무_수정 FROM STD_BD_GRD4_RESULT;")
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


# 전처리

## 중복 차대번호 제거

In [19]:
carr.shape, len(carr['차대번호'].unique())

((1906876, 11), 1906823)

In [20]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1906823, 11)

## 배출가스등급 코드 변환

In [21]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['4'], dtype=object)

## 차종 코드 변환

In [22]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '화물', '승합', '특수'], dtype=object)

## 용도 코드 변환

In [23]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [24]:
carn = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carn.shape

(1390280, 11)

## 연료 코드 변환

In [25]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['휘발유', '경유', '기타연료', 'LPG(액화석유가스)', nan, '휘발유 하이브리드',
       'CNG(압축천연가스)', '알코올', '등유', '전기', '태양열', '경유 하이브리드', 'LPG 하이브리드',
       '수소', 'LNG(액화천연가스)', 'CNG 하이브리드'], dtype=object)

## 차종유형 코드 변환

In [26]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '중형', '소형', '경형', nan], dtype=object)

## 등록&제원 병합

In [27]:
# 0.7s
cs = carn.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1390280, 17)

In [28]:
cs.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,제작일자,연료,차종유형,제작사명,차명,자동차형식,엔진형식
0,KNCSE06428K344169,4374025000,4,NaN,N,A0110003113533107,2008,화물,개인용,20220923.00,20080721.00,경유,소형,기아자동차(주),봉고Ⅲ 1톤,SEE42J-A,J3
1,KMFZSX7JABU684137,4374025000,4,NaN,N,A0810006209153110,2011,화물,개인용,20220923.00,20101022.00,경유,소형,현대자동차(주),포터Ⅱ(PORTERⅡ),HRTJA-DX-B,D4CB
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407.00,20090316.00,경유,중형,현대자동차(주),마이티군물자수송트럭,HD25-9MT0-LHFbF,D4GA
3,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909.00,20090312.00,경유,소형,기아자동차(주),봉고Ⅲ 1톤,SET42K-A,J3
4,KNCSE01429K388695,2626010300,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210902.00,20090317.00,경유,소형,기아자동차(주),봉고Ⅲ 1톤,SET42K-A,J3


## 정기&정밀 병합

In [29]:
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [30]:
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [31]:
jgtr['정기정밀'] = '정기'
eetr['정기정밀'] = '정밀'

In [32]:
# 39.6s
ins = pd.concat([jgtr, eetr], ignore_index=True)
ins.shape

(168095776, 12)

### 최근 검사만 활용

In [33]:
# 4m 14.3s
insm = ins.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
insm.shape

(34641431, 12)

## 검사종류 코드 변환

In [34]:
# 27.5s
knd_dict = {
    'A21P01':'정밀검사', 
    'A1904':'정기검사', 
    'A21T06':'종합검사(경과)', 
    'A21T05':'종합검사', 
    'A21S01':'특정검사', 
    'A1902':'재검사(2부재검사)',
    'A21R02':'2부재검사(종합)', 
    'A21P02':'정밀검사(경과)', 
    'A1903':'정기(경과)검사', 
    'A21S04':'특정재검사(배출)', 
    'A21E01':'배출재검사', 
    'A21R01':'1부재검사(종합)',
    'A21C02':'구조변경검사', 
    'A21002':'관능재검사(임시)', 
    'A21S03':'특정재검사(관능)', 
    'A21S02':'특정검사(경과)', 
    'A1901':'재검사(1부재검사)', 
    'A21T02':'종합(정밀)',
    'A21E02':'배출재검사(구변)', 
    'A21T01':'종합(경과:정밀)', 
    'A21T04':'종합(정밀) 2부재검사', 
    'A21001':'관능재검사', 
    'A21T03':'종합(정밀) 1부재검사', 
    'A21TMP':'임시검사', 
}
insm['검사종류'] = insm['검사종류'].replace(knd_dict)
insm['검사종류'].unique()

array(['종합검사', '1부재검사(종합)', '종합검사(경과)', '2부재검사(종합)', '정기검사', '재검사(1부재검사)',
       '재검사(2부재검사)', '정기(경과)검사', nan, '배출재검사(구변)', '구조변경검사',
       '종합(정밀) 1부재검사', '종합(정밀) 2부재검사', '종합(정밀)', '종합(경과:정밀)', '정밀검사',
       '특정재검사(배출)', '임시검사', '배출재검사', '관능재검사', '특정검사', '특정재검사(관능)',
       '정밀검사(경과)', '특정검사(경과)', 'A21R05', 'A21R06', '관능재검사(임시)', 'A21E03'],
      dtype=object)

## 검사방법 코드 변환

In [35]:
# 14.9s
mth_dict = {
    'A18A':'무부하검사(TSI)', 
    'A18B':'무부하검사(급가속)', 
    'A18C':'무부하검사(정지가동)',
    'A2301':'무부하검사(급가속)', 
    'A2302':'무부하검사(정지가동)',
    'A2303':'무부하검사(TSI)', 
    'A2304':'부하검사(LUG DOWN)', 
    'A2305':'부하검사(ASM-Idling)', 
    'A2306':'부하검사(KD-147)', 
    'A2307':'부하검사(ASM2525)', 
}
insm['검사방법'] = insm['검사방법'].replace(mth_dict)
insm['검사방법'].unique()

array(['부하검사(ASM-Idling)', '부하검사(KD-147)', '무부하검사(TSI)', '무부하검사(급가속)',
       '부하검사(LUG DOWN)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

## 등록&제원&정기&정밀 병합

In [36]:
# 51.4s
csi = cs.merge(insm, on='차대번호', how='left')
csi.shape

(1390280, 28)

In [37]:
csi.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀'],
      dtype='object')

In [38]:
csi.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,검사종류,검사판정,무부하매연측정치1,무부하매연측정치2,무부하매연측정치3,무부하매연판정1,검사일자,주행거리,무부하매연허용치1,정기정밀
0,KNCSE06428K344169,4374025000,4,NaN,N,A0110003113533107,2008,화물,개인용,20220923.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMFZSX7JABU684137,4374025000,4,NaN,N,A0810006209153110,2011,화물,개인용,20220923.00,...,정기검사,Y,16.00,NaN,NaN,Y,20230623.00,223409.00,20.00,정기
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407.00,...,정기(경과)검사,Y,5.00,NaN,NaN,Y,20230523.00,79425.00,20.00,정기
3,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909.00,...,2부재검사(종합),Y,14.00,NaN,NaN,Y,20220810.00,50589.00,15.00,정밀
4,KNCSE01429K388695,2626010300,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210902.00,...,종합검사(경과),Y,7.00,NaN,NaN,Y,20221205.00,36669.00,15.00,정밀


## 등록&제원&정기&정밀&법정동코드 병합

In [39]:
csi.shape

(1390280, 28)

In [40]:
csi['법정동코드'].isnull().sum()

0

In [41]:
csi['법정동코드'] = csi['법정동코드'].astype('str')
csi['법정동코드'] = csi['법정동코드'].str[:5] + '00000'
csi['법정동코드'] = pd.to_numeric(csi['법정동코드'])

In [42]:
df = csi.merge(coder, on='법정동코드', how='left')

In [43]:
df['시도'].isnull().sum()

0

## rh 법정동코드 참고하여 법정동코드 수정

In [44]:
df['법정동코드_수정'] = df['법정동코드'].copy()

## 4등급 result 파일 참고하여 DPF유무 수정

In [45]:
rdf = df.copy()

In [46]:
len(set(rdf['차대번호'].unique()) - set(rs['차대번호'].unique()))

2092

In [47]:
rs['DPF유무_수정'].value_counts(dropna=False)

무       879481
NaN     348422
유       261897
확인불가     12435
Name: DPF유무_수정, dtype: int64

In [48]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

(1502213, 2)

In [49]:
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [50]:
rdf1['DPF유무_수정'].value_counts(dropna=False)

무       806991
NaN     337658
유       234291
확인불가     11340
Name: DPF유무_수정, dtype: int64

In [51]:
df = rdf1.copy()

In [52]:
df.shape

(1390280, 32)

## 연료 확인

In [53]:
df['연료'].value_counts(dropna=False)

경유             1052802
휘발유             313038
LPG(액화석유가스)      24355
NaN                 49
기타연료                34
CNG(압축천연가스)          1
알코올                  1
Name: 연료, dtype: int64

## 연료 컬럼 추가

In [54]:
df.loc[df['연료'] == '경유', 'fuel'] = '경유'
df.loc[(df['연료'] == '휘발유') | (df['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'

# 분석

## EG 분류

In [55]:
df['fuel'].value_counts(dropna=False)

경유        1052802
휘발유_가스     337393
NaN            85
Name: fuel, dtype: int64

In [56]:
grade_list = []
for f, y, cy, e in tqdm(df[['fuel', '제작일자', '차량연식', 'DPF유무_수정']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 1390280/1390280 [00:03<00:00, 435414.54it/s]


1390280

In [57]:
df['EG'] = grade_list

In [58]:
df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_수정', 'DPF유무_수정', 'fuel', 'EG'],
      dtype='object')

# STD_BD_GRD4_CAR_CURSTT

## 4등급차만 추출

In [59]:
# 2.8s
df1 = df[df['배출가스등급'] == '4'].reset_index(drop=True)
df1.shape

(1390280, 34)

In [60]:
# 동일 배번의 연료 다양한지 확인
for one in df1['배출가스인증번호'].unique():
    if len(df1.loc[df1['배출가스인증번호'] == one, '연료'].unique()) > 1:
        print(one)

6MY-HD-24-12
LMY-KM-07
PMY-HD-17
SMY-KM-04
RMY-HD-19
TMY-HD-26
KMY-HD-05


In [61]:
df1.loc[df1['배출가스인증번호'] == '6MY-HD-24-12', '연료'].unique()

array(['경유', '휘발유'], dtype=object)

In [62]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20231026'

In [63]:
# df1['기준연월'] = '2022.12'
df1['기준연월'] = today_date[:4] + '.' + today_date[4:6]

In [64]:
df1['테이블생성일자'] = today_date

In [65]:
df1.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_수정', 'DPF유무_수정', 'fuel', 'EG', '기준연월', '테이블생성일자'],
      dtype='object')

In [66]:
df1['법정동코드_수정'] = df1['법정동코드_수정'].astype('str')
df1['법정동코드_수정'].head()

0    4374000000
1    4374000000
2    4518000000
3    2771000000
4    2626000000
Name: 법정동코드_수정, dtype: object

In [67]:
STD_BD_GRD4_CAR_CURSTT = df1[[
    '테이블생성일자', 
    '기준연월', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도',
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구',
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '배출가스등급', 
    'DPF유무_수정',
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '무부하매연측정치1', 
    '무부하매연판정1', 
    '무부하매연측정치2', 
    '무부하매연측정치3', 
    '법정동코드_수정', 
    'EG', 
    ]]
STD_BD_GRD4_CAR_CURSTT.shape

(1390280, 26)

In [68]:
STD_BD_GRD4_CAR_CURSTT.head()

,테이블생성일자,기준연월,차대번호,제원관리번호,차종,용도,차량연식,차종유형,연료,법정동코드,...,DPF유무_수정,검사방법,검사종류,검사판정,무부하매연측정치1,무부하매연판정1,무부하매연측정치2,무부하매연측정치3,법정동코드_수정,EG
0,20231026,2023.10,KNCSE06428K344169,A0110003113533107,화물,개인용,2008,소형,경유,4374000000,...,무,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4374000000,C
1,20231026,2023.10,KMFZSX7JABU684137,A0810006209153110,화물,개인용,2011,소형,경유,4374000000,...,NaN,무부하검사(급가속),정기검사,Y,16.00,Y,NaN,NaN,4374000000,X
2,20231026,2023.10,KMCGK17HP9C112030,A0810005008143209,화물,개인용,2009,중형,경유,4518000000,...,유,무부하검사(급가속),정기(경과)검사,Y,5.00,Y,NaN,NaN,4518000000,A
3,20231026,2023.10,KNCSE01429K388374,A0110003115353108,화물,개인용,2009,소형,경유,2771000000,...,무,부하검사(KD-147),2부재검사(종합),Y,14.00,Y,NaN,NaN,2771000000,C
4,20231026,2023.10,KNCSE01429K388695,A0110003115353108,화물,개인용,2009,소형,경유,2626000000,...,무,부하검사(KD-147),종합검사(경과),Y,7.00,Y,NaN,NaN,2626000000,C


In [69]:
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '기준연월':'CRTR_YM', 
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM', 
                '시군구':'SGG_NM', 
                '차명':'VHCNM', 
                '제작사명':'MNFCTR_NM', 
                '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
                '배출가스등급':'EXHST_GAS_GRD_CD_MOD',
                'DPF유무_수정':'DPF_MNTNG_YN', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1', 
                '무부하매연측정치2':'NOLOD_SMO_MEVLU2', 
                '무부하매연측정치3':'NOLOD_SMO_MEVLU3', 
                '법정동코드_수정':'STDG_CD_MOD',
                'EG':'GRD4_MLSFC',

                '제작일자':'FBCTN_YMD', 
                '차종분류':'VHCTY_CL_CD',
                }

In [70]:
STD_BD_GRD4_CAR_CURSTT = STD_BD_GRD4_CAR_CURSTT.rename(columns=ch_col_dict)
STD_BD_GRD4_CAR_CURSTT.columns

Index(['LOAD_DT', 'CRTR_YM', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2',
       'YRIDNW', 'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM',
       'VHCNM', 'MNFCTR_NM', 'EXHST_GAS_CERT_NO_MOD', 'EXHST_GAS_GRD_CD_MOD',
       'DPF_MNTNG_YN', 'INSP_MTHD', 'INSP_KND', 'INSP_JGMT',
       'NOLOD_SMO_MEVLU1', 'NOLOD_SMO_JGMT_YN1', 'NOLOD_SMO_MEVLU2',
       'NOLOD_SMO_MEVLU3', 'STDG_CD_MOD', 'GRD4_MLSFC'],
      dtype='object')

In [71]:
STD_BD_GRD4_CAR_CURSTT.shape

(1390280, 26)

## [출력] STD_BD_GRD4_CAR_CURSTT

In [72]:
# expdf = STD_BD_GRD4_CAR_CURSTT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 9s
# we.import_from_pandas(expdf, table_nm)

In [73]:
# # 0s
# STD_BD_GRD4_CAR_CURSTT.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_CAR_CURSTT.csv'), index=False)

# 분석용 테이블 생성(pass)
- STD_BD_GRD4_CAR_CURSTT_TOT
- STD_BD_GRD4_SI
- STD_BD_DAT_GRD4_CERT_NO_RVW
- STD_BD_DAT_GRD4_SI

## 1\. 경유차만 추출

In [74]:
df2 = df1[df1['연료'] == '경유'].reset_index(drop=True)

In [75]:
df2.shape

(1052802, 36)

### 차대번호 10자리 연식

In [76]:
df2['vin10'] = df2['차대번호'].str[9]

In [77]:
vin10_dict = {'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009, 'A':2010, 'B':2011, 'C':2012, 'D':2013, 'E':2014, 'F':2015, 'G':2016, 'H':2017}

In [78]:
df2['vin10_year'] = df2['vin10'].map(vin10_dict, na_action='ignore')

### 배인번호_수정 문자 타입으로 변경

In [79]:
df2['배출가스인증번호'] = df2['배출가스인증번호'].astype('str')

## 2\. 차대번호 17자리 샘플

In [80]:
df2y = df2.loc[df2['차대번호'].str.len() == 17].reset_index(drop=True)
df2n = df2.loc[df2['차대번호'].str.len() != 17].reset_index(drop=True)
df2y.shape, df2n.shape

((1052792, 38), (10, 38))

## 3\. 차대번호 연식과 차량연식 동일여부 확인

In [81]:
df3y = df2y.loc[df2y['vin10_year'] == df2y['차량연식']].reset_index(drop=True)
df3n = df2y.loc[df2y['vin10_year'] != df2y['차량연식']].reset_index(drop=True)
df3y.shape, df3n.shape

((1052708, 38), (84, 38))

## 4\. 배번, 제번, 제작사명, 차명, 검사방법별 그룹화 and 100대 초과 추출

In [82]:
df3y.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_수정', 'DPF유무_수정', 'fuel', 'EG', '기준연월', '테이블생성일자', 'vin10',
       'vin10_year'],
      dtype='object')

### 검사판정 Y만 활용

In [83]:
df3y['검사판정'].value_counts(dropna=False)

Y      987916
N       60037
NaN      4747
8           8
Name: 검사판정, dtype: int64

In [84]:
grp4 = df3y[df3y['검사판정'] == 'Y'].groupby(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호']).agg({'차대번호':'count'}).reset_index()
grp4 = grp4.rename(columns={'차대번호':'차량대수'})
grp4

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,차량대수
0,4MY-AD-20,아우디폭스바겐코리아(주),아우디 A6 3.0 TDI Quattro,무부하검사(급가속),02420000100031305,21
1,4MY-DC-33,타타대우상용차(주),대우11.5톤장축카고트럭(DPF),무부하검사(급가속),A0310006701423306,1
2,4MY-DC-33,타타대우상용차(주),대우11.5톤장축카고트럭(DPF),부하검사(LUG DOWN),A0310006701423306,5
3,4MY-KM-06,기아자동차(주),봉고Ⅲ 1톤,부하검사(KD-147),A0110003111493107,1
4,4MY-KM-06,기아자동차(주),봉고Ⅲ 냉동차,무부하검사(급가속),A0110003111783107,1
...,...,...,...,...,...,...
9710,nan,현대자동차(주),리베로(LIBERO),부하검사(KD-147),A0810003402623205,1
9711,nan,현대자동차(주),스타렉스(STAREX),부하검사(KD-147),A0810003702912205,1
9712,nan,현대자동차(주),포터Ⅱ(PORTERⅡ),무부하검사(급가속),A0810006209153110,1
9713,nan,현대자동차(주),포터Ⅱ(PORTERⅡ),부하검사(KD-147),A0810006205343107,1


In [85]:
grp4 = grp4[grp4['배출가스인증번호'] != 'nan'].reset_index(drop=True)
grp4.shape

(9230, 6)

In [86]:
grp4['차량대수'].describe()

count    9230.00
mean      106.35
std       627.24
min         1.00
25%         1.00
50%         4.00
75%        24.00
max     26513.00
Name: 차량대수, dtype: float64

### 100대 초과 샘플만 활용

In [87]:
df4 = grp4[grp4['차량대수'] > 100].reset_index(drop=True)
df4.shape

(1135, 6)

In [88]:
df4['차량대수'].sum()

890415

## 5\. 4번 조건에 해당되는 샘플만 추출

In [89]:
# 7m 43s
df5 = pd.DataFrame()
for one, two, three, four, five in df4[['배출가스인증번호', '제원관리번호', '제작사명', '차명', '검사방법']].values:
    temp = df3y[(df3y['검사판정'] == 'Y') & (df3y['배출가스인증번호'] == one) & (df3y['제원관리번호'] == two) & (df3y['제작사명'] == three) & (df3y['차명'] == four) & (df3y['검사방법'] == five)].reset_index(drop=True)
    df5 = pd.concat([df5, temp], ignore_index=True)
df5.shape

(890415, 38)

## 6\. 5번 데이터셋에서 KPI, 그리드(표), SI(산점도)용 테이블 생성

In [90]:
grp6 = df5.groupby(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호']).agg({'무부하매연측정치1':[lambda x:x.describe()['25%'], lambda x:x.describe()['50%'], lambda x:x.describe()['75%']], '차대번호':'count'}).reset_index()
grp6.columns = ['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호', 'q1', 'q2', 'q3', '차량대수']
grp6

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,0.00,0.00,0.00,136
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,0.00,0.00,0.00,130
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800031206,0.00,0.00,1.00,201
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800111207,0.00,0.00,1.00,260
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,부하검사(KD-147),02420004900031207,0.00,0.00,1.00,314
...,...,...,...,...,...,...,...,...,...
1130,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800871309,1.00,4.00,8.00,915
1131,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800901310,0.00,4.00,8.00,976
1132,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800911310,1.00,4.00,8.00,1131
1133,9MY-PA-13-1,(주)볼보자동차코리아,볼보 XC60D5,무부하검사(급가속),00920003200021309,0.00,0.00,0.00,136


In [91]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20231026'

In [92]:
grp6['테이블생성일자'] = today_date

In [93]:
STD_BD_GRD4_CAR_CURSTT_TOT = grp6[[
    '테이블생성일자',
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]]
STD_BD_GRD4_CAR_CURSTT_TOT.shape

(1135, 10)

In [94]:
chc_dict = {
    '테이블생성일자':'LOAD_DT', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '제원관리번호':'MANG_MNG_NO', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    'q1':'LOWR_QRT',
    'q2':'MID_QRT',
    'q3':'UP_QRT',
    '차량대수':'VHCL_MKCNT',

    # '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    # '차종':'VHCTY_CD', 
    # '용도':'PURPS_CD2', 
    # '차종유형':'CHCTY_TY', 
    # '법정동코드':'STDG_CD', 
    # '검사종류':'INSP_KND', 
    # '검사판정':'INSP_JGMT', 
    # '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    # '차대번호':'VIN', 
    # '등급_수정':'EXHST_GAS_GRD_CD_MOD', 
    # 'DPF유무_수정':'DPF_MNTNG_YN', 
    # '시도명':'CTPV_NM', 
    # '시군구명':'SGG_NM', 
    # '차종분류':'VHCTY_CL_CD', 
    }

In [95]:
STD_BD_GRD4_CAR_CURSTT_TOT = STD_BD_GRD4_CAR_CURSTT_TOT.rename(columns=chc_dict)
STD_BD_GRD4_CAR_CURSTT_TOT.columns

Index(['LOAD_DT', 'VHCNM', 'MNFCTR_NM', 'MANG_MNG_NO', 'EXHST_GAS_CERT_NO_MOD',
       'INSP_MTHD', 'LOWR_QRT', 'MID_QRT', 'UP_QRT', 'VHCL_MKCNT'],
      dtype='object')

### [출력] STD_BD_GRD4_CAR_CURSTT_TOT

In [96]:
# expdf = STD_BD_GRD4_CAR_CURSTT_TOT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [97]:
# # 0s
# STD_BD_GRD4_CAR_CURSTT_TOT.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_CAR_CURSTT_TOT.csv'), index=False)

In [98]:
sidf = df5[[
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '차량연식', 
    '차명', 
    '차종유형', 
    '제작사명', 
    '연료', 
    '법정동코드', 
    '배출가스인증번호', 
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '주행거리', 
    '무부하매연판정1', 
    '무부하매연허용치1', 
    '무부하매연측정치1'
    ]]
sidf.shape

(890415, 17)

In [99]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [100]:
sidf['차령'] = current_yr - sidf['차량연식']
sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_21908\4139794486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['차령'] = current_yr - sidf['차량연식']
C:\Users\kbjung\AppData\Local\Temp\ipykernel_21908\4139794486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']


In [101]:
sidf[['차량연식', '차령', 'SI']].isnull().sum()

차량연식        0
차령          0
SI      27901
dtype: int64

In [102]:
sidf['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_21908\3010636050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['테이블생성일자'] = today_date


In [103]:
sidf.shape

(890415, 20)

In [104]:
sidf1 = sidf[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '검사방법',
    '주행거리', 
    '차령',
    'SI', 
    ]]

In [105]:
chc_dict = {
    '테이블생성일자':'LOAD_DT',
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '주행거리':'DRVNG_DSTNC',
    '차령':'VHCAG',
    
    # '차종':'VHCTY_CD', 
    # '연식':'YRIDNW', 
    # '차종유형':'VHCTY_TY', 
    # '연료':'FUEL_CD',
    # '법정동코드':'STDG_CD', 
    # '검사종류':'INSP_KND', 
    # '검사판정':'INSP_JGMT', 
    # '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    # '무부하매연허용치1':'NOLOD_SMO_PRMT_VAL1',
    # '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    }

In [106]:
STD_BD_GRD4_SI = sidf1.rename(columns=chc_dict)
STD_BD_GRD4_SI.columns

Index(['LOAD_DT', 'VIN', 'MANG_MNG_NO', 'VHCNM', 'MNFCTR_NM',
       'EXHST_GAS_CERT_NO_MOD', 'INSP_MTHD', 'DRVNG_DSTNC', 'VHCAG', 'SI'],
      dtype='object')

### [출력] STD_BD_GRD4_SI

In [107]:
# expdf = STD_BD_GRD4_SI
# table_nm = 'STD_BD_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 6s
# we.import_from_pandas(expdf, table_nm)

In [108]:
# # 0s
# STD_BD_GRD4_SI.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_SI.csv'), index=False)

## 7\. 5번 데이터셋에서 DAT용 (검토구분 계산) 테이블 생성

### 검토구분(양호/주의 판정) 

In [109]:
# grp6 : df5.groupby(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호'])
grp7 = grp6.copy()

In [110]:
grp7['q2_mean'] = grp7.groupby(['배출가스인증번호', '제작사명', '차명', '검사방법'])['q2'].transform('mean')
grp7

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,0.00,0.00,0.00,136,20231026,0.00
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,0.00,0.00,0.00,130,20231026,0.00
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800031206,0.00,0.00,1.00,201,20231026,0.00
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800111207,0.00,0.00,1.00,260,20231026,0.00
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,부하검사(KD-147),02420004900031207,0.00,0.00,1.00,314,20231026,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1130,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800871309,1.00,4.00,8.00,915,20231026,4.29
1131,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800901310,0.00,4.00,8.00,976,20231026,4.29
1132,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800911310,1.00,4.00,8.00,1131,20231026,4.29
1133,9MY-PA-13-1,(주)볼보자동차코리아,볼보 XC60D5,무부하검사(급가속),00920003200021309,0.00,0.00,0.00,136,20231026,0.00


In [111]:
grp7.loc[(grp7['q2'] > grp7['q2_mean']*5) | (grp7['q2'] < grp7['q2_mean']/5), '검토구분'] = '주의'
grp7['검토구분'] = grp7['검토구분'].fillna('양호')

In [112]:
grp7['검토구분'].value_counts(dropna=False)

양호    1119
주의      16
Name: 검토구분, dtype: int64

In [113]:
grp7.head()

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean,검토구분
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,0.00,0.00,0.00,136,20231026,0.00,양호
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,0.00,0.00,0.00,130,20231026,0.00,양호
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800031206,0.00,0.00,1.00,201,20231026,0.00,양호
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800111207,0.00,0.00,1.00,260,20231026,0.00,양호
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,부하검사(KD-147),02420004900031207,0.00,0.00,1.00,314,20231026,0.00,양호


In [130]:
len(grp7.loc[grp7['검토구분'] == '양호', '배출가스인증번호'].unique())

67

In [131]:
len(grp7.loc[grp7['검토구분'] == '주의', '배출가스인증번호'].unique())

6

In [119]:
grp7.loc[grp7['검토구분'] == '주의', ['배출가스인증번호']]

,배출가스인증번호
14,5MY-HD-24-59
51,6MY-HD-14-63
52,6MY-HD-14-63
92,6MY-HD-24-100
93,6MY-HD-24-100
94,6MY-HD-24-100
184,6MY-KM-14-61
185,6MY-KM-14-61
209,6MY-MK-14-13
257,7MY-HD-13-58


In [123]:
len(grp7.loc[grp7['검토구분'] == '주의', '배출가스인증번호'].unique())

6

In [124]:
grp7.loc[grp7['검토구분'] == '주의', '배출가스인증번호'].unique()

array(['5MY-HD-24-59', '6MY-HD-14-63', '6MY-HD-24-100', '6MY-KM-14-61',
       '6MY-MK-14-13', '7MY-HD-13-58'], dtype=object)

In [120]:
grp7[grp7['배출가스인증번호'] == '5MY-HD-24-59']

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean,검토구분
10,5MY-HD-24-59,현대자동차(주),맥스크루즈 (MAXCRUZ),부하검사(KD-147),A0810009300091314,0.00,0.00,0.00,144,20231026,0.00,양호
11,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006300941205,0.00,2.00,8.00,173,20231026,1.88,양호
12,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006300961205,0.00,1.00,8.00,437,20231026,1.88,양호
13,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006300981205,0.00,1.00,8.00,642,20231026,1.88,양호
14,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301001205,0.00,0.00,6.00,106,20231026,1.88,주의
15,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301181206,0.00,3.00,12.00,283,20231026,1.88,양호
16,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301201206,0.00,3.00,11.00,967,20231026,1.88,양호
17,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301221206,0.00,2.00,10.00,955,20231026,1.88,양호
18,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301241206,0.00,3.00,11.00,168,20231026,1.88,양호
19,5MY-HD-24-59,현대자동차(주),투싼(TUCSON),부하검사(KD-147),A0810006300941205,0.00,1.00,4.00,842,20231026,1.25,양호


In [125]:
grp7[grp7['배출가스인증번호'] == '6MY-HD-14-63']

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean,검토구분
48,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301431207,0.00,2.00,8.00,139,20231026,1.25,양호
49,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301451207,0.00,1.00,7.00,490,20231026,1.25,양호
50,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301471207,0.00,1.00,8.00,434,20231026,1.25,양호
51,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301701207,0.00,0.00,6.00,403,20231026,1.25,주의
52,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301721207,0.00,0.00,3.00,289,20231026,1.25,주의
53,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301941208,0.00,1.00,8.00,196,20231026,1.25,양호
54,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006301961208,0.00,2.50,9.00,140,20231026,1.25,양호
55,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006302181208,0.00,1.00,7.00,343,20231026,1.25,양호
56,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006302201208,0.00,2.00,8.00,648,20231026,1.25,양호
57,6MY-HD-14-63,현대자동차(주),투싼(TUCSON),무부하검사(급가속),A0810006302221208,0.00,2.00,8.00,420,20231026,1.25,양호


In [126]:
grp7[grp7['배출가스인증번호'] == '6MY-HD-24-100']

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean,검토구분
90,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800461306,6.00,15.00,25.00,422,20231026,7.45,양호
91,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800481306,4.50,13.00,24.50,547,20231026,7.45,양호
92,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800561306,0.00,0.00,3.00,232,20231026,7.45,주의
93,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800581306,0.00,0.00,4.00,457,20231026,7.45,주의
94,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800591306,0.00,0.00,1.00,222,20231026,7.45,주의
95,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800681307,3.00,13.50,23.75,298,20231026,7.45,양호
96,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800811207,2.00,9.00,19.00,611,20231026,7.45,양호
97,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800831207,2.00,8.00,18.00,641,20231026,7.45,양호
98,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800901307,3.50,8.00,16.00,107,20231026,7.45,양호
99,6MY-HD-24-100,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006800921307,3.00,8.00,15.00,505,20231026,7.45,양호


In [127]:
grp7[grp7['배출가스인증번호'] == '6MY-KM-14-61']

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean,검토구분
169,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400351205,0.00,1.00,4.00,123,20231026,2.20,양호
170,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400361205,0.00,2.00,9.00,1046,20231026,2.20,양호
171,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400431206,0.00,3.00,12.00,592,20231026,2.20,양호
172,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400441206,0.00,3.00,10.00,2633,20231026,2.20,양호
173,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400491207,0.00,1.00,7.00,173,20231026,2.20,양호
174,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400501207,0.00,1.00,7.00,1215,20231026,2.20,양호
175,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400571208,0.00,1.00,9.00,101,20231026,2.20,양호
176,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400581208,0.00,3.00,9.00,1249,20231026,2.20,양호
177,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400661209,0.00,3.00,9.00,141,20231026,2.20,양호
178,6MY-KM-14-61,기아자동차(주),스포티지,무부하검사(급가속),A0110003400681209,0.00,4.00,10.00,1970,20231026,2.20,양호


In [128]:
grp7[grp7['배출가스인증번호'] == '6MY-MK-14-13']

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean,검토구분
209,6MY-MK-14-13,메르세데스벤츠코리아(주),E220 CDI,부하검사(KD-147),00820001800111308,0.00,0.00,1.00,124,20231026,0.50,주의
210,6MY-MK-14-13,메르세데스벤츠코리아(주),E220 CDI,부하검사(KD-147),00820001900001306,0.00,1.00,4.00,224,20231026,0.50,양호


In [129]:
grp7[grp7['배출가스인증번호'] == '7MY-HD-13-58']

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean,검토구분
257,7MY-HD-13-58,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006801001307,0.00,0.00,2.00,355,20231026,0.14,주의
258,7MY-HD-13-58,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006801021307,0.00,0.00,1.00,623,20231026,0.14,주의
259,7MY-HD-13-58,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006801031307,0.00,0.00,1.00,185,20231026,0.14,주의
260,7MY-HD-13-58,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006801141308,0.00,1.00,4.00,200,20231026,0.14,주의
261,7MY-HD-13-58,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006801371308,0.00,0.00,4.00,348,20231026,0.14,주의
262,7MY-HD-13-58,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006801381308,0.00,0.00,2.00,966,20231026,0.14,주의
263,7MY-HD-13-58,현대자동차(주),싼타페(SANTAFE),무부하검사(급가속),A0810006801391308,0.00,0.00,3.00,146,20231026,0.14,주의


In [ ]:
cert_no_list, pf_list = [], []
for cert_no in grp7['배출가스인증번호'].unique():
    temp = grp7[(grp7['배출가스인증번호'] == cert_no) & (grp7['검토구분'] == '주의')]
    if temp.shape[0] == 0:
        pf = '양호'
    else:
        pf = '주의'
    cert_no_list.append(cert_no)
    pf_list.append(pf)
cert_pf = pd.DataFrame({'배출가스인증번호':cert_no_list, '검토주의':pf_list})
cert_pf

,배출가스인증번호,검토주의
0,5MY-AD-13-06,양호
1,5MY-AD-14-08,양호
2,5MY-AD-14-18,양호
3,5MY-BK-14-07,양호
4,5MY-HD-24-35,양호
...,...,...
63,9MY-HD-14-21,양호
64,9MY-KM-14-22,양호
65,9MY-KM-14-41,양호
66,9MY-PA-13-1,양호


In [ ]:
cert_pf['검토주의'].value_counts(dropna=False)

검토주의
양호    62
주의     6
Name: count, dtype: int64

In [ ]:
grp7.columns

Index(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호', 'q1', 'q2', 'q3', '차량대수',
       '테이블생성일자', 'q2_mean', '검토구분'],
      dtype='object')

In [ ]:
STD_BD_DAT_GRD4_CERT_NO_RVW = grp7[[
    '배출가스인증번호',
    '검사방법',
    '검토구분',
    '제작사명',
    '차명',
    '제원관리번호',
    'q1',
    'q2',
    'q3',
    '테이블생성일자',
]]

In [ ]:
cdict = {
    '배출가스인증번호':'EXHST_GAS_CERT_NO',
    '검사방법':'INSP_MTHD',
    '검토구분':'RVW_SE',
    '제작사명':'RPRS_MNFCTR_NM',
    '차명':'RPRS_VHCNM', 
    '제원관리번호':'MANG_MNG_NO',
    'q1':'LOWR_QRT',
    'q2':'MID_QRT',
    'q3':'UP_QRT',
    '테이블생성일자':'LOAD_DT',
}
STD_BD_DAT_GRD4_CERT_NO_RVW = STD_BD_DAT_GRD4_CERT_NO_RVW.rename(columns=cdict)
STD_BD_DAT_GRD4_CERT_NO_RVW.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RVW_SE', 'RPRS_MNFCTR_NM',
       'RPRS_VHCNM', 'MANG_MNG_NO', 'LOWR_QRT', 'MID_QRT', 'UP_QRT',
       'LOAD_DT'],
      dtype='object')

In [ ]:
# # 0s
# STD_BD_DAT_GRD4_CERT_NO_RVW.to_csv(os.path.join(df1_fold, f'STD_BD_DAT_GRD4_CERT_NO_RVW({today_date}).csv'), index=False)

### [출력] STD_BD_DAT_GRD4_CERT_NO_RVW

In [ ]:
# expdf = STD_BD_DAT_GRD4_CERT_NO_RVW
# table_nm = 'STD_BD_DAT_GRD4_CERT_NO_RVW'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 0s
# STD_BD_DAT_GRD4_CERT_NO_RVW.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_CERT_NO_RVW.csv'), index=False)

In [ ]:
# 조건에 맞는 샘플 df에 '검토구분' 정보 추가
# sidf.merge(grp7, on=['배출가스인증번호', '제원관리번호', '제작사명', '차명', '검사방법'])

In [ ]:
sidf.shape

(891036, 20)

In [ ]:
sidf.columns

Index(['차대번호', '제원관리번호', '차종', '차량연식', '차명', '차종유형', '제작사명', '연료', '법정동코드',
       '배출가스인증번호', '검사방법', '검사종류', '검사판정', '주행거리', '무부하매연판정1', '무부하매연허용치1',
       '무부하매연측정치1', '차령', 'SI', '테이블생성일자'],
      dtype='object')

In [ ]:
grp7.shape

(1141, 12)

In [ ]:
grp7.columns

Index(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호', 'q1', 'q2', 'q3', '차량대수',
       '테이블생성일자', 'q2_mean', '검토구분'],
      dtype='object')

In [ ]:
grp_sidf = sidf.groupby(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호']).agg({'차종':lambda x:x.value_counts().index[0], '연료':lambda x:x.value_counts().index[0], '차량연식':lambda x : x.nsmallest(1), 'SI':'mean'}).reset_index()
grp_sidf = grp_sidf.rename(columns={'차량연식':'최초연식', 'SI':'열화도'})
grp_sidf

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,차종,연료,최초연식,열화도
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,승용,경유,2006,0.03
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,승용,경유,2007,0.02
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800031206,승용,경유,2007,0.04
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800111207,승용,경유,2008,0.04
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,부하검사(KD-147),02420004900031207,승용,경유,2008,0.09
...,...,...,...,...,...,...,...,...,...
1136,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800871309,승용,경유,2010,0.31
1137,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800901310,승용,경유,2011,0.33
1138,9MY-KM-14-41,기아자동차(주),카니발,부하검사(KD-147),A0110003800911310,승용,경유,2011,0.33
1139,9MY-PA-13-1,(주)볼보자동차코리아,볼보 XC60D5,무부하검사(급가속),00920003200021309,승용,경유,2010,0.02


In [ ]:
grp_sidf.head(10)

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,차종,연료,최초연식,열화도
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,승용,경유,2006,0.03
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,승용,경유,2007,0.02
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800031206,승용,경유,2007,0.04
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800111207,승용,경유,2008,0.04
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,부하검사(KD-147),02420004900031207,승용,경유,2008,0.09
5,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,부하검사(KD-147),02420004900041208,승용,경유,2009,0.10
6,5MY-AD-14-18,아우디폭스바겐코리아(주),Jetta 2.0 TDI,부하검사(KD-147),02420007000021207,승용,경유,2008,0.06
7,5MY-BK-14-07,비엠더블유코리아(주),BMW X3 3.0d,무부하검사(급가속),00120001600041305,승용,경유,2006,0.05
8,5MY-HD-24-35,현대자동차(주),베르나(VERNA),부하검사(KD-147),A0810006700341105,승용,경유,2006,0.47
9,5MY-HD-24-35,현대자동차(주),베르나(VERNA),부하검사(KD-147),A0810006700421105,승용,경유,2006,0.48


In [ ]:
df71 = grp_sidf.merge(grp7[['배출가스인증번호', '제원관리번호', '제작사명', '차명', '검사방법', '검토구분']], on=['배출가스인증번호', '제원관리번호', '제작사명', '차명', '검사방법'], how='left')
df71.shape

(1141, 10)

In [ ]:
df71.columns

Index(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호', '차종', '연료', '최초연식', '열화도',
       '검토구분'],
      dtype='object')

In [ ]:
df71['검토구분'].value_counts(dropna=False)

검토구분
양호    1125
주의      16
Name: count, dtype: int64

In [ ]:
df71.head()

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,차종,연료,최초연식,열화도,검토구분
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,승용,경유,2006,0.03,양호
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,승용,경유,2007,0.02,양호
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800031206,승용,경유,2007,0.04,양호
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,부하검사(KD-147),02420006800111207,승용,경유,2008,0.04,양호
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,부하검사(KD-147),02420004900031207,승용,경유,2008,0.09,양호


In [ ]:
df71['테이블생성일자'] = today_date

In [ ]:
STD_BD_DAT_GRD4_SI = df71[[
    '배출가스인증번호',
    '검사방법', 
    '검토구분',
    '제작사명',
    '차명',
    '차종', 
    '연료',
    '최초연식',
    '열화도',
    '테이블생성일자'
]]

In [ ]:
# cdict = {
#     '배출가스인증번호':'EXHST_GAS_CERT_NO', 
#     '검토구분':'RVW_SE', 
#     '대표제작사명':'RPRS_MNFCTR_NM', 
#     '대표차명':'RPRS_VHCNM', 
#     '대표차종':'RPRS_VHCTY_CD', 
#     '대표차연료':'RPRS_FUEL', 
#     '최초연식':'FRST_YRIDNW', 
#     '열화도':'SI', 
#     '테이블생성일자':'LOAD_DT', 
#     # '검사방법':'INSP_MTHD', 
# }
cdict = {
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '검사방법':'INSP_MTHD', 
    '검토구분':'RVW_SE', 
    '제작사명':'RPRS_MNFCTR_NM', 
    '차명':'RPRS_VHCNM', 
    '차종':'RPRS_VHCTY_CD', 
    '연료':'RPRS_FUEL', 
    '최초연식':'FRST_YRIDNW', 
    '열화도':'SI', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_SI = STD_BD_DAT_GRD4_SI.rename(columns=cdict)
STD_BD_DAT_GRD4_SI.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RVW_SE', 'RPRS_MNFCTR_NM',
       'RPRS_VHCNM', 'RPRS_VHCTY_CD', 'RPRS_FUEL', 'FRST_YRIDNW', 'SI',
       'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_GRD4_SI

In [ ]:
# expdf = STD_BD_DAT_GRD4_SI
# table_nm = 'STD_BD_DAT_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 0s
# STD_BD_DAT_GRD4_SI.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_SI.csv'), index=False)

In [ ]:
df5.shape

(891036, 38)

In [ ]:
df5.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_수정', 'DPF유무_수정', 'fuel', 'EG', '기준연월', '테이블생성일자', 'vin10',
       'vin10_year'],
      dtype='object')

In [ ]:
STD_BD_GRD4_CAR_CURSTT_MOD = df5[[
    '테이블생성일자', 
    '차대번호', 
    '배출가스인증번호', 
    '제원관리번호', 
    '제작사명', 
    '차명', 
    '검사방법', 
    '무부하매연측정치1', 
]]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT',
    '차대번호':'VIN', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '제원관리번호':'MANG_MNG_NO', 
    '제작사명':'MNFCTR_NM', 
    '차명':'VHCNM', 
    '검사방법':'INSP_MTHD', 
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
}
STD_BD_GRD4_CAR_CURSTT_MOD = STD_BD_GRD4_CAR_CURSTT_MOD.rename(columns=cdict)
STD_BD_GRD4_CAR_CURSTT_MOD.columns         

Index(['LOAD_DT', 'VIN', 'EXHST_GAS_CERT_NO', 'MANG_MNG_NO', 'MNFCTR_NM',
       'VHCNM', 'INSP_MTHD', 'NOLOD_SMO_MEVLU1'],
      dtype='object')

## [출력] STD_BD_GRD4_CAR_CURSTT_MOD

In [ ]:
# expdf = STD_BD_GRD4_CAR_CURSTT_MOD
# table_nm = 'STD_BD_GRD4_CAR_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 9s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 0s
# STD_BD_GRD4_CAR_CURSTT_MOD.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_CAR_CURSTT_MOD.csv'), index=False)

# backup

# 분석

## 열화계수(SI) 지수 계산
- SI = 측정치 / 허용치

In [ ]:
df3y.shape

(1052707, 38)

In [ ]:
df3y.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_수정', 'DPF유무_수정', 'fuel', 'EG', '기준연월', '테이블생성일자', 'vin10',
       'vin10_year'],
      dtype='object')

In [ ]:
sidf = df3y[[
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '차량연식', 
    '차명', 
    '차종유형', 
    '제작사명', 
    '연료', 
    '법정동코드', 
    '배출가스인증번호', 
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '주행거리', 
    '무부하매연판정1', 
    '무부하매연허용치1', 
    '무부하매연측정치1'
    ]]
sidf.shape

(1052707, 17)

In [ ]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [ ]:
sidf['차령'] = current_yr - sidf['차량연식']
sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_18100\4139794486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['차령'] = current_yr - sidf['차량연식']
C:\Users\kbjung\AppData\Local\Temp\ipykernel_18100\4139794486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']


In [ ]:
sidf['차량연식'].isnull().sum()

0

In [ ]:
sidf[['차령', 'SI']].isnull().sum()

차령        0
SI    46433
dtype: int64

In [ ]:
sidf[['무부하매연측정치1', '무부하매연허용치1']].isnull().sum()

무부하매연측정치1    12545
무부하매연허용치1    12546
dtype: int64

In [ ]:
sidf.head()

,차대번호,제원관리번호,차종,차량연식,차명,차종유형,제작사명,연료,법정동코드,배출가스인증번호,검사방법,검사종류,검사판정,주행거리,무부하매연판정1,무부하매연허용치1,무부하매연측정치1,차령,SI
0,KMFZSX7JABU684137,A0810006209153110,화물,2011,포터Ⅱ(PORTERⅡ),소형,현대자동차(주),경유,4374000000,nan,무부하검사(급가속),정기검사,Y,223409.00,Y,20.00,16.00,12,0.80
1,KNCSE06428K344169,A0110003113533107,화물,2008,봉고Ⅲ 1톤,소형,기아자동차(주),경유,4374000000,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN
2,KMCGK17HP9C112030,A0810005008143209,화물,2009,마이티군물자수송트럭,중형,현대자동차(주),경유,4518000000,7MY-HD-24-28,무부하검사(급가속),정기(경과)검사,Y,79425.00,Y,20.00,5.00,14,0.25
3,KNCSE01429K388374,A0110003115353108,화물,2009,봉고Ⅲ 1톤,소형,기아자동차(주),경유,2771000000,7MY-KM-14-65,부하검사(KD-147),2부재검사(종합),Y,50589.00,Y,15.00,14.00,14,0.93
4,KNFSE0A439K387582,A0110003116073108,화물,2009,봉고Ⅲ 냉동차,소형,기아자동차(주),경유,3020000000,7MY-KM-14-65,부하검사(KD-147),종합검사,Y,151328.00,Y,15.00,0.00,14,0.00


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230823'

In [ ]:
sidf['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_18100\3010636050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['테이블생성일자'] = today_date


In [ ]:
sidf1 = sidf[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '검사방법',
    '주행거리', 
    '차령',
    'SI', 
    ]]

In [ ]:
chc_dict = {
    '테이블생성일자':'LOAD_DT',
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차종':'VHCTY_CD', 
    '연식':'YRIDNW', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '차종유형':'VHCTY_TY', 
    '연료':'FUEL_CD',
    '법정동코드':'STDG_CD', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '검사종류':'INSP_KND', 
    '검사판정':'INSP_JGMT', 
    '주행거리':'DRVNG_DSTNC',
    '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    '무부하매연허용치1':'NOLOD_SMO_PRMT_VAL1',
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '차령':'VHCAG',
    }

In [ ]:
STD_BD_GRD4_SI = sidf1.rename(columns=chc_dict)
STD_BD_GRD4_SI.columns

Index(['LOAD_DT', 'VIN', 'MANG_MNG_NO', 'VHCNM', 'MNFCTR_NM',
       'EXHST_GAS_CERT_NO_MOD', 'INSP_MTHD', 'DRVNG_DSTNC', 'VHCAG', 'SI'],
      dtype='object')

In [ ]:
STD_BD_GRD4_SI.head()

,LOAD_DT,VIN,MANG_MNG_NO,VHCNM,MNFCTR_NM,EXHST_GAS_CERT_NO_MOD,INSP_MTHD,DRVNG_DSTNC,VHCAG,SI
0,20230823,KMFZSX7JABU684137,A0810006209153110,포터Ⅱ(PORTERⅡ),현대자동차(주),nan,무부하검사(급가속),223409.00,12,0.80
1,20230823,KNCSE06428K344169,A0110003113533107,봉고Ⅲ 1톤,기아자동차(주),nan,NaN,NaN,15,NaN
2,20230823,KMCGK17HP9C112030,A0810005008143209,마이티군물자수송트럭,현대자동차(주),7MY-HD-24-28,무부하검사(급가속),79425.00,14,0.25
3,20230823,KNCSE01429K388374,A0110003115353108,봉고Ⅲ 1톤,기아자동차(주),7MY-KM-14-65,부하검사(KD-147),50589.00,14,0.93
4,20230823,KNFSE0A439K387582,A0110003116073108,봉고Ⅲ 냉동차,기아자동차(주),7MY-KM-14-65,부하검사(KD-147),151328.00,14,0.00


In [ ]:
STD_BD_GRD4_SI.shape

(1052707, 10)

### [출력] SI 지수 정보(STD_BD_GRD4_SI)

In [ ]:
# expdf = STD_BD_GRD4_SI
# table_nm = 'STD_BD_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 6s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 0s
# STD_BD_GRD4_SI.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_SI.csv'), index=False)

## 동일 배번에서 제번별 매연 boxplot

In [ ]:
total_g_df1 = total_g_df.loc[total_g_df['배출가스인증번호'] != 'nan'] # !!! 수정(2023.08.23)
total_g_df1.shape

(27359, 13)

In [ ]:
len(total_g_df1['배출가스인증번호'].unique())

200

In [ ]:
sample01 = total_g_df1.loc[total_g_df1['대수'] > 100].reset_index(drop=True)
sample01.shape

(1498, 13)

In [ ]:
len(total_g_df1['배출가스인증번호'].unique())

200

### quantile

#### !!! 확인중...

In [ ]:
# 22s
temp = df3y.groupby(['배출가스인증번호', '검사종류', '검사방법', '제원관리번호']).agg({'무부하매연측정치1':[ lambda x:x.describe()['25%'], lambda x:x.describe()['50%'], lambda x:x.describe()['75%'] ], '차대번호':'count'}).reset_index()
temp

배출가스인증번호        검사종류            검사방법             제원관리번호  무부하매연측정치1   
                                                                <lambda_0>   
0      4MY-AD-20   2부재검사(종합)      무부하검사(급가속)  02420000100031305      27.75  \
1      4MY-AD-20  재검사(2부재검사)      무부하검사(급가속)  02420000100031305      19.00   
2      4MY-AD-20        정기검사      무부하검사(급가속)  02420000100031305      16.25   
3      4MY-AD-20        종합검사      무부하검사(급가속)  02420000100031305       3.75   
4      4MY-AD-20    종합검사(경과)      무부하검사(급가속)  02420000100031305      14.00   
...          ...         ...             ...                ...        ...   
22576        nan    종합검사(경과)  부하검사(LUG DOWN)  B4T10005300003308       8.00   
22577        nan    종합검사(경과)  부하검사(LUG DOWN)  B4V10002400004307       2.00   
22578        nan    종합검사(경과)  부하검사(LUG DOWN)  B8E10002400003306      10.00   
22579        nan    종합검사(경과)  부하검사(LUG DOWN)  B8W20002200003308       9.00   
22580        nan    종합검사(경과)  부하검사(LUG DOWN)  B9G10003400003307       5.00   

                             차대번호  
      <lambda_1> <lambda_2> count  
0          29.50      31.25     2  
1          19.00      19.00     1  
2          23.00      25.25     4  
3           9.50      14.50    12  
4          17.00      17.00     3  
...          ...        ...   ...  
22576       8.00       8.00     1  
22577       2.00       2.00     1  
22578      10.00      10.00     1  
22579       9.00       9.00     1  
22580       5.00       5.00     1  

[22581 rows x 8 columns]

In [ ]:
temp.columns = ['배출가스인증번호', '검사종류', '검사방법', '제원관리번호', 'q1', 'q2', 'q3', '차량대수']
temp

,배출가스인증번호,검사종류,검사방법,제원관리번호,q1,q2,q3,차량대수
0,4MY-AD-20,2부재검사(종합),무부하검사(급가속),02420000100031305,27.75,29.50,31.25,2
1,4MY-AD-20,재검사(2부재검사),무부하검사(급가속),02420000100031305,19.00,19.00,19.00,1
2,4MY-AD-20,정기검사,무부하검사(급가속),02420000100031305,16.25,23.00,25.25,4
3,4MY-AD-20,종합검사,무부하검사(급가속),02420000100031305,3.75,9.50,14.50,12
4,4MY-AD-20,종합검사(경과),무부하검사(급가속),02420000100031305,14.00,17.00,17.00,3
...,...,...,...,...,...,...,...,...
22576,nan,종합검사(경과),부하검사(LUG DOWN),B4T10005300003308,8.00,8.00,8.00,1
22577,nan,종합검사(경과),부하검사(LUG DOWN),B4V10002400004307,2.00,2.00,2.00,1
22578,nan,종합검사(경과),부하검사(LUG DOWN),B8E10002400003306,10.00,10.00,10.00,1
22579,nan,종합검사(경과),부하검사(LUG DOWN),B8W20002200003308,9.00,9.00,9.00,1


In [ ]:
temp = temp[temp['배출가스인증번호'] != 'nan'].reset_index(drop=True)
temp.shape

(21744, 8)

In [ ]:
temp2 = temp[temp['차량대수'] > 100].reset_index(drop=True)
temp2.shape

(1454, 8)

In [ ]:
temp2.head(10)

,배출가스인증번호,검사종류,검사방법,제원관리번호,q1,q2,q3,차량대수
0,5MY-AD-13-06,종합검사,무부하검사(급가속),02420004800011305,0.00,0.00,0.00,116
1,5MY-AD-14-08,종합검사,부하검사(KD-147),02420006800031206,0.00,0.00,1.00,181
2,5MY-AD-14-08,종합검사,부하검사(KD-147),02420006800111207,0.00,0.00,0.00,231
3,5MY-AD-14-18,종합검사,부하검사(KD-147),02420004900031207,0.00,0.00,1.00,279
4,5MY-AD-14-18,종합검사,부하검사(KD-147),02420004900041208,0.00,0.00,1.00,450
5,5MY-AD-14-18,종합검사,부하검사(KD-147),02420007000021207,0.00,0.00,1.00,122
6,5MY-HD-24-35,종합검사,부하검사(KD-147),A0810006700341105,6.50,13.00,21.50,168
7,5MY-HD-24-35,종합검사,부하검사(KD-147),A0810006700421105,7.00,11.00,22.00,137
8,5MY-HD-24-59,1부재검사(종합),부하검사(KD-147),A0810006300981205,0.00,1.00,5.00,108
9,5MY-HD-24-59,1부재검사(종합),부하검사(KD-147),A0810006301201206,0.00,2.00,5.00,154


In [ ]:
temp2.drop_duplicates(['배출가스인증번호', '제원관리번호']).shape

(716, 8)

In [ ]:
temp2['dup'] = temp2.duplicated(['배출가스인증번호', '제원관리번호'])
temp2.head()

,배출가스인증번호,검사종류,검사방법,제원관리번호,q1,q2,q3,차량대수,dup
0,5MY-AD-13-06,종합검사,무부하검사(급가속),02420004800011305,0.00,0.00,0.00,116,False
1,5MY-AD-14-08,종합검사,부하검사(KD-147),02420006800031206,0.00,0.00,1.00,181,False
2,5MY-AD-14-08,종합검사,부하검사(KD-147),02420006800111207,0.00,0.00,0.00,231,False
3,5MY-AD-14-18,종합검사,부하검사(KD-147),02420004900031207,0.00,0.00,1.00,279,False
4,5MY-AD-14-18,종합검사,부하검사(KD-147),02420004900041208,0.00,0.00,1.00,450,False


In [ ]:
temp2.loc[temp2['dup'] == True, '배출가스인증번호'].unique()

array(['5MY-HD-24-59', '5MY-KM-24-17', '6MY-DW-14-18', '6MY-HD-14-63',
       '6MY-HD-14-64', '6MY-HD-14-97', '6MY-HD-24-100', '6MY-HD-24-12',
       '6MY-HD-24-44', '6MY-KM-14-57', '6MY-KM-14-61', '6MY-KM-24-81',
       '7MY-DW-13-07', '7MY-HD-13-58', '7MY-HD-14-45', '7MY-HD-14-46',
       '7MY-HD-14-47', '7MY-HD-14-57', '7MY-HD-14-68', '7MY-HD-24-08',
       '7MY-HD-24-09', '7MY-HD-24-28', '7MY-KM-14-63', '7MY-KM-14-64',
       '7MY-KM-14-65', '7MY-SM-14-36', '7MY-SY-14-24', '7MY-SY-14-34',
       '7MY-SY-14-49', '7MY-SY-14-60', '8MY-KM-14-29', '9MY-HD-14-21',
       '9MY-KM-14-22', '9MY-KM-14-41'], dtype=object)

In [ ]:
temp2[temp2['배출가스인증번호'] == '5MY-HD-24-59'].sort_values(['검사종류', '검사방법', '제원관리번호'])

,배출가스인증번호,검사종류,검사방법,제원관리번호,q1,q2,q3,차량대수,dup
8,5MY-HD-24-59,1부재검사(종합),부하검사(KD-147),A0810006300981205,0.00,1.00,5.00,108,False
9,5MY-HD-24-59,1부재검사(종합),부하검사(KD-147),A0810006301201206,0.00,2.00,5.00,154,False
10,5MY-HD-24-59,1부재검사(종합),부하검사(KD-147),A0810006301221206,0.00,2.00,5.00,182,False
11,5MY-HD-24-59,재검사(1부재검사),무부하검사(급가속),A0810006301201206,0.00,0.00,0.00,146,True
12,5MY-HD-24-59,재검사(1부재검사),무부하검사(급가속),A0810006301221206,0.00,0.00,0.00,138,True
13,5MY-HD-24-59,재검사(2부재검사),무부하검사(급가속),A0810006301221206,0.00,0.00,5.75,102,True
14,5MY-HD-24-59,정기검사,무부하검사(급가속),A0810006300941205,0.00,3.00,11.00,157,False
15,5MY-HD-24-59,정기검사,무부하검사(급가속),A0810006300961205,0.00,3.00,10.00,367,False
16,5MY-HD-24-59,정기검사,무부하검사(급가속),A0810006300981205,0.00,3.00,10.00,562,True
17,5MY-HD-24-59,정기검사,무부하검사(급가속),A0810006301181206,1.00,6.00,16.00,249,False


In [ ]:
df1[(df1['배출가스인증번호'] == '5MY-HD-24-59') & (df1['검사종류'] == '정기검사') & (df1['검사방법'] == '무부하검사(급가속)') & (df1['제원관리번호'] == 'A0810006300981205')].shape

(562, 36)

In [ ]:
# 4m 11.2s
boxplot_df = pd.DataFrame()
quantile_df = pd.DataFrame()
for one in sample01['배출가스인증번호'].unique(): # 통계에서 100대 초과인 배인번호만 추출
    temp_one = df3y.loc[(df3y['배출가스인증번호'] == one) & (df3y['검사판정'] == 'Y')].reset_index(drop=True) # 해당 배인번호 중 검사판정이 'Y'인 샘플만 추출
    if temp_one.shape[0] > 100: # 100대 초과면 진행
        for two in temp_one['검사종류'].unique(): # 검사종류별 샘플 추출
            temp_two = temp_one.loc[temp_one['검사종류'] == two].reset_index(drop=True)
            if temp_two.shape[0] > 100:
                for three in temp_two['검사방법'].unique(): # 검사방법별 샘플 추출
                    temp_three = temp_two.loc[temp_two['검사방법'] == three].reset_index(drop=True)
                    boxplot_df = pd.concat([boxplot_df, temp_three], ignore_index=True) # 해당 샘플만 추출하여 쌓기
                    if temp_three.shape[0] > 100:
                        xticks_list= []
                        data_list = []
                        for four in temp_three['제원관리번호'].unique():
                            temp_four = temp_three.loc[temp_three['제원관리번호'] == four].reset_index(drop=True).dropna(subset=['무부하매연측정치1'])
                            if temp_four.shape[0] > 100:
                                xticks_list.append(four)
                                data_list.append(temp_four['무부하매연측정치1'])
                                temp_four['q1'] = temp_four['무부하매연측정치1'].describe()['25%']
                                temp_four['q2'] = temp_four['무부하매연측정치1'].describe()['50%']
                                temp_four['q3'] = temp_four['무부하매연측정치1'].describe()['75%']
                                temp_four['차량대수'] = temp_four.shape[0]
                                quantile_df = pd.concat([quantile_df, temp_four], ignore_index=True) # 제번별 4분위 값 df형태로 저장

In [ ]:
quantile_df.shape

(794428, 42)

In [ ]:
quantile_df[['배출가스인증번호', '제원관리번호', '검사종류', '검사방법', 'q1', 'q2', 'q3', '차량대수']].head(10)

,배출가스인증번호,제원관리번호,검사종류,검사방법,q1,q2,q3,차량대수
0,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
1,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
2,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
3,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
4,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
5,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
6,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
7,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
8,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332
9,7MY-HD-24-28,A0810005008503209,종합검사,부하검사(LUG DOWN),5.00,8.00,11.00,332


In [ ]:
quantile_df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_수정', 'DPF유무_수정', 'fuel', 'EG', '기준연월', '테이블생성일자', 'vin10',
       'vin10_year', 'q1', 'q2', 'q3', '차량대수'],
      dtype='object')

In [ ]:
len(quantile_df['배출가스인증번호'].unique())

63

In [ ]:
quantile_df1 = quantile_df.drop_duplicates(['배출가스인증번호', '제원관리번호', '검사방법', '검사종류'])
quantile_df1.shape

(1391, 42)

In [ ]:
quantile_df1.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,fuel,EG,기준연월,테이블생성일자,vin10,vin10_year,q1,q2,q3,차량대수
0,KMFGA17HPAC152609,4719000000,4,7MY-HD-24-28,N,A0810005008503209,2010,화물,개인용,20200128.00,...,경유,A,2022.12,20230823,A,2010.00,5.00,8.00,11.00,332
332,KMFGA17HPAC141581,4812100000,4,7MY-HD-24-28,N,A0810005008603209,2010,화물,개인용,20171226.00,...,경유,A,2022.12,20230823,A,2010.00,4.00,8.00,11.00,512
844,KMFGA17HPBC187773,1150000000,4,7MY-HD-24-28,N,A0810005009343210,2011,화물,영업용,20111216.00,...,경유,A,2022.12,20230823,B,2011.00,2.00,6.00,9.00,476
1320,KMJHD17HPAC008298,4719000000,4,7MY-HD-24-28,N,A0810005603012209,2010,승합,개인용,20101231.00,...,경유,A,2022.12,20230823,A,2010.00,3.00,8.00,11.00,327
1647,KMFGA17HPAC152695,4141000000,4,7MY-HD-24-28,N,A0810005008583209,2010,화물,영업용,20101130.00,...,경유,A,2022.12,20230823,A,2010.00,3.00,7.00,10.75,650


In [ ]:
quantile_df1.sort_values(['배출가스인증번호', '제원관리번호'])

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,fuel,EG,기준연월,테이블생성일자,vin10,vin10_year,q1,q2,q3,차량대수
794313,WVWZZZ3DZ68005835,4812700000,4,5MY-AD-13-06,N,02420004800011305,2006,승용,개인용,20070109.00,...,경유,D,2022.12,20230823,6,2006.00,0.00,0.00,0.00,115
732562,WVWZZZ3CZ7P166019,4477000000,4,5MY-AD-14-08,N,02420006800031206,2007,승용,개인용,20071024.00,...,경유,D,2022.12,20230823,7,2007.00,0.00,0.00,1.00,179
732333,WVWZZZ3CZ8P070420,4139000000,4,5MY-AD-14-08,N,02420006800111207,2008,승용,개인용,20080905.00,...,경유,C,2022.12,20230823,8,2008.00,0.00,0.00,0.00,229
586639,WVWZZZ1KZ8W338454,4513000000,4,5MY-AD-14-18,N,02420004900031207,2008,승용,개인용,20081007.00,...,경유,C,2022.12,20230823,8,2008.00,0.00,0.00,1.00,279
586067,WVWZZZ1KZ9W146549,2635000000,4,5MY-AD-14-18,N,02420004900041208,2009,승용,개인용,20090526.00,...,경유,C,2022.12,20230823,9,2009.00,0.00,0.00,1.00,450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394570,KNAMN816BBS464259,4122000000,4,9MY-KM-14-41,N,A0110003800911310,2011,승용,개인용,20111202.00,...,경유,C,2022.12,20230823,B,2011.00,2.00,5.00,8.25,112
398289,KNAMN816BBS464236,4163000000,4,9MY-KM-14-41,N,A0110003800911310,2011,승용,개인용,20130626.00,...,경유,C,2022.12,20230823,B,2011.00,0.00,4.00,8.00,893
439402,KNAMN816BBS464253,4817000000,4,9MY-KM-14-41,N,A0110003800911310,2011,승용,개인용,20120220.00,...,경유,C,2022.12,20230823,B,2011.00,2.00,6.00,10.00,155
789184,YV1DZ7046A2105590,4143000000,4,9MY-PA-13-1,N,00920003200021309,2010,승용,개인용,20100423.00,...,경유,A,2022.12,20230823,A,2010.00,0.00,0.00,0.00,111


In [ ]:
quantile_df1.groupby(['배출가스인증번호', '제원관리번호']).agg({'q2':'mean'})

q2
배출가스인증번호     제원관리번호                
5MY-AD-13-06 02420004800011305 0.00
5MY-AD-14-08 02420006800031206 0.00
             02420006800111207 0.00
5MY-AD-14-18 02420004900031207 0.00
             02420004900041208 0.00
...                             ...
nan          00620000800081209 4.00
             00620000900021210 5.00
             00920002300051307 0.00
             00920002300141308 1.00
             A0110003113613107 6.00

[708 rows x 1 columns]

In [ ]:
quantile_df1.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구',
       '법정동코드_수정', 'DPF유무_수정', 'fuel', 'EG', '기준연월', '테이블생성일자', 'vin10',
       'vin10_year', 'q1', 'q2', 'q3', '차량대수'],
      dtype='object')

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230821'

In [ ]:
quantile_df1['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_14124\752261507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantile_df1['테이블생성일자'] = today_date


In [ ]:
STD_BD_GRD4_CAR_CURSTT_TOT = quantile_df1[[
    '테이블생성일자',
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]]
STD_BD_GRD4_CAR_CURSTT_TOT.shape

(1401, 10)

In [ ]:
chc_dict = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '차종':'VHCTY_CD', 
    '용도':'PURPS_CD2', 
    '차종유형':'CHCTY_TY', 
    '법정동코드':'STDG_CD', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '검사종류':'INSP_KND', 
    '검사판정':'INSP_JGMT', 
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    'q1':'LOWR_QRT',
    'q2':'MID_QRT',
    'q3':'UP_QRT',
    '차량대수':'VHCL_MKCNT',
    # '등급_수정':'EXHST_GAS_GRD_CD_MOD', 
    # 'DPF유무_수정':'DPF_MNTNG_YN', 
    # '시도명':'CTPV_NM', 
    # '시군구명':'SGG_NM', 
    # '차종분류':'VHCTY_CL_CD', 
    }

In [ ]:
STD_BD_GRD4_CAR_CURSTT_TOT = STD_BD_GRD4_CAR_CURSTT_TOT.rename(columns=chc_dict)
STD_BD_GRD4_CAR_CURSTT_TOT.columns

Index(['LOAD_DT', 'VHCNM', 'MNFCTR_NM', 'MANG_MNG_NO', 'EXHST_GAS_CERT_NO_MOD',
       'INSP_MTHD', 'LOWR_QRT', 'MID_QRT', 'UP_QRT', 'VHCL_MKCNT'],
      dtype='object')

In [ ]:
STD_BD_GRD4_CAR_CURSTT_TOT.shape

(1401, 10)

In [ ]:
STD_BD_GRD4_CAR_CURSTT_TOT.head()

,LOAD_DT,VHCNM,MNFCTR_NM,MANG_MNG_NO,EXHST_GAS_CERT_NO_MOD,INSP_MTHD,LOWR_QRT,MID_QRT,UP_QRT,VHCL_MKCNT
0,20230821,3008 1.6 HDi MCP,한불모터스(주),00620000900021210,nan,부하검사(KD-147),0.00,5.00,10.00,184
184,20230821,308 1.6 HDi MCP,한불모터스(주),00620000800081209,nan,부하검사(KD-147),1.00,4.00,9.50,159
343,20230821,Nouvelle Peugeot 308SW HDi,한불모터스(주),00620000800021208,nan,부하검사(KD-147),3.00,6.00,9.00,109
452,20230821,푸조 NEW 307SW HDi,한불모터스(주),00620000400221207,nan,부하검사(KD-147),2.00,6.00,9.25,112
564,20230821,푸조 407 HDi FAP A6,한불모터스(주),00620000500251207,nan,부하검사(KD-147),1.00,4.00,9.00,160


In [ ]:
set(STD_BD_GRD4_CAR_CURSTT_TOT['MNFCTR_NM'].unique()) - set(STD_BD_GRD4_CAR_CURSTT['MNFCTR_NM'].unique())

set()

In [ ]:
set(STD_BD_GRD4_SI['MNFCTR_NM'].unique()) - set(STD_BD_GRD4_CAR_CURSTT['MNFCTR_NM'].unique())

set()

### [출력] STD_BD_GRD4_CAR_CURSTT_TOT

In [ ]:
# expdf = STD_BD_GRD4_CAR_CURSTT_TOT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 0s
# STD_BD_GRD4_CAR_CURSTT_TOT.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_CAR_CURSTT_TOT.csv'), index=False)

## 인증번호검토

In [ ]:
cert_df1 = quantile_df1[[
    '테이블생성일자',
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]].reset_index(drop=True)
cert_df1.shape

(1391, 10)

In [ ]:
cert_df1.head()

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수
0,20230823,이-마이티,현대자동차(주),A0810005008503209,7MY-HD-24-28,부하검사(LUG DOWN),5.00,8.00,11.00,332
1,20230823,이-마이티,현대자동차(주),A0810005008603209,7MY-HD-24-28,부하검사(LUG DOWN),4.00,8.00,11.00,512
2,20230823,이-마이티,현대자동차(주),A0810005009343210,7MY-HD-24-28,부하검사(LUG DOWN),2.00,6.00,9.00,476
3,20230823,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,8.00,11.00,327
4,20230823,이-마이티,현대자동차(주),A0810005008583209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,7.00,10.75,650


In [ ]:
sample01.columns

Index(['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법', '검사판정',
       '대수', 'mean', 'min', 'max', '합격률(%)'],
      dtype='object')

In [ ]:
# cert_df2 = cert_df1[cert_df1['배출가스인증번호'] != 'nan'].reset_index(drop=True)
cert_df2 = cert_df1[cert_df1['배출가스인증번호'].isnull() == False].reset_index(drop=True)
cert_df2.shape

(1391, 10)

In [ ]:
# cert_df1['q2_mean'] = cert_df1.groupby(['배출가스인증번호', '검사방법'])['q2'].transform('mean')
cert_df2['q2_mean'] = cert_df2.groupby(['제작사명', '차명', '배출가스인증번호', '검사방법'])['q2'].transform('mean')
cert_df2

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean
0,20230823,이-마이티,현대자동차(주),A0810005008503209,7MY-HD-24-28,부하검사(LUG DOWN),5.00,8.00,11.00,332,6.57
1,20230823,이-마이티,현대자동차(주),A0810005008603209,7MY-HD-24-28,부하검사(LUG DOWN),4.00,8.00,11.00,512,6.57
2,20230823,이-마이티,현대자동차(주),A0810005009343210,7MY-HD-24-28,부하검사(LUG DOWN),2.00,6.00,9.00,476,6.57
3,20230823,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,8.00,11.00,327,6.50
4,20230823,이-마이티,현대자동차(주),A0810005008583209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,7.00,10.75,650,6.57
...,...,...,...,...,...,...,...,...,...,...,...
1386,20230823,싼타페(SANTAFE),현대자동차(주),A0810006800761307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,1.00,105,0.00
1387,20230823,싼타페(SANTAFE),현대자동차(주),A0810006800781307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,3.00,223,0.00
1388,20230823,쎄라토,기아자동차(주),A0110003001211206,5MY-KM-24-64,부하검사(KD-147),4.00,7.00,14.00,205,8.00
1389,20230823,쎄라토,기아자동차(주),A0110003000981205,5MY-KM-24-64,부하검사(KD-147),5.00,9.00,14.75,262,8.00


In [ ]:
cert_df2.loc[(cert_df2['q2'] > cert_df2['q2_mean']*5) | (cert_df2['q2'] < cert_df2['q2_mean']/5), '검토구분'] = '주의'
cert_df2.loc[cert_df2['q3'] == 0, '검토구분'] = '양호'
cert_df2['검토구분'] = cert_df2['검토구분'].fillna('양호')

In [ ]:
cert_df2.loc[cert_df2['q3'] == 0, '검토구분'].value_counts()

검토구분
양호    102
Name: count, dtype: int64

In [ ]:
cert_df2['검토구분'].value_counts()

검토구분
양호    1340
주의      51
Name: count, dtype: int64

In [ ]:
grp1 = cert_df2.groupby(['배출가스인증번호', '검사방법', '검토구분', '제원관리번호']).agg({'제작사명':lambda x:x.value_counts().index[0], '차명':lambda x:x.value_counts().index[0]}).reset_index()
grp1 = grp1.rename(columns={'제작사명':'대표제작사명', '차명':'대표차명'})
grp1

,배출가스인증번호,검사방법,검토구분,제원관리번호,대표제작사명,대표차명
0,5MY-AD-13-06,무부하검사(급가속),양호,02420004800011305,아우디폭스바겐코리아(주),Phaeton 3.0 TDI
1,5MY-AD-14-08,부하검사(KD-147),양호,02420006800031206,아우디폭스바겐코리아(주),Passat 2.0 TDI
2,5MY-AD-14-08,부하검사(KD-147),양호,02420006800111207,아우디폭스바겐코리아(주),Passat 2.0 TDI
3,5MY-AD-14-18,부하검사(KD-147),양호,02420004900031207,아우디폭스바겐코리아(주),Golf 2.0 TDI
4,5MY-AD-14-18,부하검사(KD-147),양호,02420004900041208,아우디폭스바겐코리아(주),Golf 2.0 TDI
...,...,...,...,...,...,...
985,9MY-KM-14-41,부하검사(KD-147),양호,A0110003800871309,기아자동차(주),카니발
986,9MY-KM-14-41,부하검사(KD-147),양호,A0110003800901310,기아자동차(주),카니발
987,9MY-KM-14-41,부하검사(KD-147),양호,A0110003800911310,기아자동차(주),카니발
988,9MY-PA-13-1,무부하검사(급가속),양호,00920003200021309,(주)볼보자동차코리아,볼보 XC60D5


In [ ]:
cg1 = cert_df2.merge(grp1, on=['배출가스인증번호', '검사방법', '검토구분', '제원관리번호'], how='left')
cg1.shape

(1391, 14)

In [ ]:
cg1

,테이블생성일자,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean,검토구분,대표제작사명,대표차명
0,20230823,이-마이티,현대자동차(주),A0810005008503209,7MY-HD-24-28,부하검사(LUG DOWN),5.00,8.00,11.00,332,6.57,양호,현대자동차(주),이-마이티
1,20230823,이-마이티,현대자동차(주),A0810005008603209,7MY-HD-24-28,부하검사(LUG DOWN),4.00,8.00,11.00,512,6.57,양호,현대자동차(주),이-마이티
2,20230823,이-마이티,현대자동차(주),A0810005009343210,7MY-HD-24-28,부하검사(LUG DOWN),2.00,6.00,9.00,476,6.57,양호,현대자동차(주),이-마이티
3,20230823,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,8.00,11.00,327,6.50,양호,현대자동차(주),이-카운티
4,20230823,이-마이티,현대자동차(주),A0810005008583209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,7.00,10.75,650,6.57,양호,현대자동차(주),이-마이티
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386,20230823,싼타페(SANTAFE),현대자동차(주),A0810006800761307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,1.00,105,0.00,양호,현대자동차(주),싼타페(SANTAFE)
1387,20230823,싼타페(SANTAFE),현대자동차(주),A0810006800781307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,3.00,223,0.00,양호,현대자동차(주),싼타페(SANTAFE)
1388,20230823,쎄라토,기아자동차(주),A0110003001211206,5MY-KM-24-64,부하검사(KD-147),4.00,7.00,14.00,205,8.00,양호,기아자동차(주),쎄라토
1389,20230823,쎄라토,기아자동차(주),A0110003000981205,5MY-KM-24-64,부하검사(KD-147),5.00,9.00,14.75,262,8.00,양호,기아자동차(주),쎄라토


In [ ]:
cg1['검토구분'].unique()

array(['양호', '주의'], dtype=object)

In [152]:
cg1[['검토구분', '대표제작사명', '대표차명']].isnull().sum()

검토구분      0
대표제작사명    0
대표차명      0
dtype: int64

In [153]:
cg2 = cg1.drop_duplicates(['배출가스인증번호', '검사방법', '검토구분', '대표제작사명', '대표차명', '제원관리번호']).reset_index(drop=True)
cg2.shape

(990, 14)

In [154]:
cg2['검토구분'].value_counts(dropna=False)

검토구분
양호    942
주의     48
Name: count, dtype: int64

In [155]:
list(cg2.columns)

['테이블생성일자',
 '차명',
 '제작사명',
 '제원관리번호',
 '배출가스인증번호',
 '검사방법',
 'q1',
 'q2',
 'q3',
 '차량대수',
 'q2_mean',
 '검토구분',
 '대표제작사명',
 '대표차명']

In [163]:
STD_BD_DAT_GRD4_CERT_NO_RVW = cg2[[
    '배출가스인증번호',
    '검사방법',
    '검토구분',
    '대표제작사명',
    '대표차명',
    '제원관리번호',
    'q1',
    'q2',
    'q3',
    '테이블생성일자',
]]

In [164]:
cdict = {
    '배출가스인증번호':'EXHST_GAS_CERT_NO',
    '검사방법':'INSP_MTHD',
    '검토구분':'RVW_SE',
    '대표제작사명':'RPRS_MNFCTR_NM',
    '대표차명':'RPRS_VHCNM', 
    '제원관리번호':'MANG_MNG_NO',
    'q1':'LOWR_QRT',
    'q2':'MID_QRT',
    'q3':'UP_QRT',
    '테이블생성일자':'LOAD_DT',
}
STD_BD_DAT_GRD4_CERT_NO_RVW = STD_BD_DAT_GRD4_CERT_NO_RVW.rename(columns=cdict)
STD_BD_DAT_GRD4_CERT_NO_RVW.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RVW_SE', 'RPRS_MNFCTR_NM',
       'RPRS_VHCNM', 'MANG_MNG_NO', 'LOWR_QRT', 'MID_QRT', 'UP_QRT',
       'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_GRD4_CERT_NO_RVW

In [165]:
# expdf = STD_BD_DAT_GRD4_CERT_NO_RVW
# table_nm = 'STD_BD_DAT_GRD4_CERT_NO_RVW'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [166]:
# # 0s
# STD_BD_DAT_GRD4_CERT_NO_RVW.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_CERT_NO_RVW.csv'), index=False)

## 열화도 테이블

In [167]:
sidf.columns

Index(['차대번호', '제원관리번호', '차종', '차량연식', '차명', '차종유형', '제작사명', '연료', '법정동코드',
       '배출가스인증번호', '검사방법', '검사종류', '검사판정', '주행거리', '무부하매연판정1', '무부하매연허용치1',
       '무부하매연측정치1', '차령', 'SI', '테이블생성일자'],
      dtype='object')

In [168]:
sidf['차량연식'].nsmallest(1)

1052704    1996
Name: 차량연식, dtype: int64

In [169]:
sidf.groupby(['배출가스인증번호', '검사방법']).agg({'차량연식':lambda x : x.nsmallest(1)}).reset_index()

,배출가스인증번호,검사방법,차량연식
0,4MY-AD-20,무부하검사(급가속),2006
1,4MY-DC-33,무부하검사(급가속),2007
2,4MY-DC-33,부하검사(LUG DOWN),2007
3,4MY-KM-06,무부하검사(급가속),2008
4,4MY-KM-06,부하검사(KD-147),2008
...,...,...,...
379,C135,부하검사(KD-147),2008
380,nan,무부하검사(TSI),1996
381,nan,무부하검사(급가속),2002
382,nan,부하검사(KD-147),2005


In [238]:
# grp2 = sidf.groupby(['배출가스인증번호', '검사방법']).agg({'제작사명':lambda x:x.value_counts().index[0], '차명':lambda x:x.value_counts().index[0], '차종':lambda x:x.value_counts().index[0], '연료':lambda x:x.value_counts().index[0], '차량연식':lambda x : x.nsmallest(1), 'SI':'mean'}).reset_index()
grp2 = sidf.groupby(['배출가스인증번호']).agg({'제작사명':lambda x:x.value_counts().index[0], '차명':lambda x:x.value_counts().index[0], '차종':lambda x:x.value_counts().index[0], '연료':lambda x:x.value_counts().index[0], '차량연식':lambda x : x.nsmallest(1), 'SI':'mean'}).reset_index()
grp2 = grp2.rename(columns={'제작사명':'대표제작사명', '차명':'대표차명', '차종':'대표차종', '연료':'대표차연료', '차량연식':'최초연식', 'SI':'열화도'})
grp2

,배출가스인증번호,대표제작사명,대표차명,대표차종,대표차연료,최초연식,열화도
0,4MY-AD-20,아우디폭스바겐코리아(주),아우디 A6 3.0 TDI Quattro,승용,경유,2006,0.37
1,4MY-DC-33,타타대우상용차(주),대우11.5톤장축카고트럭(DPF),화물,경유,2007,0.34
2,4MY-KM-06,기아자동차(주),봉고Ⅲ 일반덤프,화물,경유,2008,0.36
3,4MY-PA-06,(주)볼보자동차코리아,프리랜더 2.0D,승용,경유,2005,0.38
4,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,승용,경유,2005,0.03
...,...,...,...,...,...,...,...
196,9MY-PP-14-333,(주)비오에프인베스트먼트,랜드로버 디펜더,승용,경유,2009,0.12
197,9MY-PP-14-409,로버코리아(주),랜드로버 디펜더110,승용,경유,2009,0.25
198,AMY-JL-13-9,(주)재규어랜드로버코리아,프리랜더 TD4,승용,경유,2009,0.01
199,C135,쌍용자동차(주),액티언,승용,경유,2008,1.05


In [239]:
cg1.columns

Index(['테이블생성일자', '차명', '제작사명', '제원관리번호', '배출가스인증번호', '검사방법', 'q1', 'q2', 'q3',
       '차량대수', 'q2_mean', '검토구분', '대표제작사명', '대표차명'],
      dtype='object')

In [156]:
cert_df1 = quantile_df1[[
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]]
cert_df1.shape

(1391, 9)

In [157]:
cert_df1.head()

,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수
0,이-마이티,현대자동차(주),A0810005008503209,7MY-HD-24-28,부하검사(LUG DOWN),5.00,8.00,11.00,332
332,이-마이티,현대자동차(주),A0810005008603209,7MY-HD-24-28,부하검사(LUG DOWN),4.00,8.00,11.00,512
844,이-마이티,현대자동차(주),A0810005009343210,7MY-HD-24-28,부하검사(LUG DOWN),2.00,6.00,9.00,476
1320,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,8.00,11.00,327
1647,이-마이티,현대자동차(주),A0810005008583209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,7.00,10.75,650


In [158]:
# cert_df2 = cert_df1[cert_df1['배출가스인증번호'] != 'nan'].reset_index(drop=True)
cert_df2 = cert_df1[cert_df1['배출가스인증번호'].isnull() == False].reset_index(drop=True)
cert_df2.shape

(1391, 9)

In [159]:
# cert_df1['q2_mean'] = cert_df1.groupby(['배출가스인증번호', '검사방법'])['q2'].transform('mean')
cert_df2['q2_mean'] = cert_df2.groupby(['제작사명', '차명', '배출가스인증번호', '검사방법'])['q2'].transform('mean')
cert_df2

,차명,제작사명,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean
0,이-마이티,현대자동차(주),A0810005008503209,7MY-HD-24-28,부하검사(LUG DOWN),5.00,8.00,11.00,332,6.57
1,이-마이티,현대자동차(주),A0810005008603209,7MY-HD-24-28,부하검사(LUG DOWN),4.00,8.00,11.00,512,6.57
2,이-마이티,현대자동차(주),A0810005009343210,7MY-HD-24-28,부하검사(LUG DOWN),2.00,6.00,9.00,476,6.57
3,이-카운티,현대자동차(주),A0810005603012209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,8.00,11.00,327,6.50
4,이-마이티,현대자동차(주),A0810005008583209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,7.00,10.75,650,6.57
...,...,...,...,...,...,...,...,...,...,...
1386,싼타페(SANTAFE),현대자동차(주),A0810006800761307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,1.00,105,0.00
1387,싼타페(SANTAFE),현대자동차(주),A0810006800781307,7MY-HD-23-01,무부하검사(급가속),0.00,0.00,3.00,223,0.00
1388,쎄라토,기아자동차(주),A0110003001211206,5MY-KM-24-64,부하검사(KD-147),4.00,7.00,14.00,205,8.00
1389,쎄라토,기아자동차(주),A0110003000981205,5MY-KM-24-64,부하검사(KD-147),5.00,9.00,14.75,262,8.00


In [160]:
cert_df2.loc[(cert_df2['q2'] > cert_df2['q2_mean']*5) | (cert_df2['q2'] < cert_df2['q2_mean']/5), '검토구분'] = '주의'
cert_df2.loc[cert_df2['q3'] == 0, '검토구분'] = '양호'
cert_df2['검토구분'] = cert_df2['검토구분'].fillna('양호')

In [161]:
cert_df2['검토구분'].value_counts()

검토구분
양호    1340
주의      51
Name: count, dtype: int64

In [162]:
sidf.shape

(1052707, 20)

In [163]:
sidf.columns

Index(['차대번호', '제원관리번호', '차종', '차량연식', '차명', '차종유형', '제작사명', '연료', '법정동코드',
       '배출가스인증번호', '검사방법', '검사종류', '검사판정', '주행거리', '무부하매연판정1', '무부하매연허용치1',
       '무부하매연측정치1', '차령', 'SI', '테이블생성일자'],
      dtype='object')

In [164]:
sidf.groupby(['배출가스인증번호', '검사방법']).agg({'차량연식':lambda x : x.nsmallest(1)}).reset_index()

,배출가스인증번호,검사방법,차량연식
0,4MY-AD-20,무부하검사(급가속),2006
1,4MY-DC-33,무부하검사(급가속),2007
2,4MY-DC-33,부하검사(LUG DOWN),2007
3,4MY-KM-06,무부하검사(급가속),2008
4,4MY-KM-06,부하검사(KD-147),2008
...,...,...,...
378,C135,부하검사(KD-147),2008
379,nan,무부하검사(TSI),1996
380,nan,무부하검사(급가속),2002
381,nan,부하검사(KD-147),2005


In [165]:
# grp2 = sidf.groupby(['배출가스인증번호', '검사방법']).agg({'제작사명':lambda x:x.value_counts().index[0], '차명':lambda x:x.value_counts().index[0], '차종':lambda x:x.value_counts().index[0], '연료':lambda x:x.value_counts().index[0], '차량연식':lambda x : x.nsmallest(1), 'SI':'mean'}).reset_index()
grp2 = sidf.groupby(['배출가스인증번호', '검사방법']).agg({'제작사명':lambda x:x.value_counts().index[0], '차명':lambda x:x.value_counts().index[0], '차종':lambda x:x.value_counts().index[0], '연료':lambda x:x.value_counts().index[0], '차량연식':lambda x : x.nsmallest(1), 'SI':'mean'}).reset_index()
grp2 = grp2.rename(columns={'차량연식':'최초연식', 'SI':'열화도'})
grp2
# '제작사명':'대표제작사명', '차명':'대표차명', '차종':'대표차종', '연료':'대표차연료',

,배출가스인증번호,검사방법,제작사명,차명,차종,연료,최초연식,열화도
0,4MY-AD-20,무부하검사(급가속),아우디폭스바겐코리아(주),아우디 A6 3.0 TDI Quattro,승용,경유,2006,0.34
1,4MY-DC-33,무부하검사(급가속),타타대우상용차(주),대우11.5톤장축카고트럭(DPF),화물,경유,2007,0.80
2,4MY-DC-33,부하검사(LUG DOWN),타타대우상용차(주),대우11.5톤장축카고트럭(DPF),화물,경유,2007,0.32
3,4MY-KM-06,무부하검사(급가속),기아자동차(주),봉고Ⅲ 일반덤프,화물,경유,2008,0.40
4,4MY-KM-06,부하검사(KD-147),기아자동차(주),봉고Ⅲ 일반덤프,화물,경유,2008,0.35
...,...,...,...,...,...,...,...,...
378,C135,부하검사(KD-147),쌍용자동차(주),액티언,승용,경유,2008,1.26
379,nan,무부하검사(TSI),쌍용자동차(주),코란도KJ,승용,경유,1996,0.00
380,nan,무부하검사(급가속),에프씨에이코리아(주),짚 랭글러,승용,경유,2002,0.15
381,nan,부하검사(KD-147),한불모터스(주),볼보 S80D5,승용,경유,2005,0.28


In [166]:
grp2 = grp2.drop_duplicates(['제작사명', '차명', '배출가스인증번호', '검사방법']).reset_index(drop=True)
grp2.shape

(383, 8)

In [167]:
cert_df2.columns

Index(['차명', '제작사명', '제원관리번호', '배출가스인증번호', '검사방법', 'q1', 'q2', 'q3', '차량대수',
       'q2_mean', '검토구분'],
      dtype='object')

In [168]:
cert_df2.shape

(1391, 11)

In [185]:
grp2.columns

Index(['배출가스인증번호', '검사방법', '제작사명', '차명', '차종', '연료', '최초연식', '열화도'], dtype='object')

In [186]:
cg2 = cert_df2[['제원관리번호', '배출가스인증번호', '검사방법', 'q1', 'q2', 'q3', '차량대수','q2_mean', '검토구분']].merge(grp2, on=['배출가스인증번호', '검사방법'], how='left')
cg2.shape

(1391, 15)

In [187]:
cg2.head()

,제원관리번호,배출가스인증번호,검사방법,q1,q2,q3,차량대수,q2_mean,검토구분,제작사명,차명,차종,연료,최초연식,열화도
0,A0810005008503209,7MY-HD-24-28,부하검사(LUG DOWN),5.00,8.00,11.00,332,6.57,양호,현대자동차(주),이-마이티,화물,경유,2007,0.47
1,A0810005008603209,7MY-HD-24-28,부하검사(LUG DOWN),4.00,8.00,11.00,512,6.57,양호,현대자동차(주),이-마이티,화물,경유,2007,0.47
2,A0810005009343210,7MY-HD-24-28,부하검사(LUG DOWN),2.00,6.00,9.00,476,6.57,양호,현대자동차(주),이-마이티,화물,경유,2007,0.47
3,A0810005603012209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,8.00,11.00,327,6.50,양호,현대자동차(주),이-마이티,화물,경유,2007,0.47
4,A0810005008583209,7MY-HD-24-28,부하검사(LUG DOWN),3.00,7.00,10.75,650,6.57,양호,현대자동차(주),이-마이티,화물,경유,2007,0.47


In [189]:
cert_no_list, pf_list = [], []
for cert_no in cg2['배출가스인증번호'].unique():
    temp = cg2[(cg2['배출가스인증번호'] == cert_no) & (cg2['검토구분'] == '주의')]
    if temp.shape[0] == 0:
        pf = '양호'
    else:
        pf = '주의'
    cert_no_list.append(cert_no)
    pf_list.append(pf)
cert_pf = pd.DataFrame({'배출가스인증번호':cert_no_list, '검토주의':pf_list})
cert_pf

,배출가스인증번호,검토주의
0,7MY-HD-24-28,양호
1,7MY-KM-14-65,양호
2,7MY-KM-14-64,양호
3,7MY-HD-24-30,양호
4,7MY-HD-14-68,주의
...,...,...
58,7MY-SY-14-61,양호
59,7MY-SY-14-49,양호
60,7MY-HD-23-01,양호
61,5MY-KM-24-64,양호


In [191]:
cert_pf['검토주의'].value_counts(dropna=False)

검토주의
양호    56
주의     7
Name: count, dtype: int64

In [188]:
cg2['검토구분'].value_counts(dropna=False)

검토구분
양호    1340
주의      51
Name: count, dtype: int64

In [172]:
cg2[['차종', '연료', '최초연식']].isnull().sum()

차종      0
연료      0
최초연식    0
dtype: int64

In [173]:
cg2['테이블생성일자'] = today_date

In [174]:
list(cg2.columns)

['차명',
 '제작사명',
 '제원관리번호',
 '배출가스인증번호',
 '검사방법',
 'q1',
 'q2',
 'q3',
 '차량대수',
 'q2_mean',
 '검토구분',
 '차종',
 '연료',
 '최초연식',
 '열화도',
 '테이블생성일자']

In [175]:
STD_BD_DAT_GRD4_SI = cg2[[
    '배출가스인증번호',
    '검사방법', 
    '검토구분',
    '제작사명',
    '차명',
    '차종', 
    '연료',
    '최초연식',
    '열화도',
    '테이블생성일자'
]]

In [176]:
# cdict = {
#     '배출가스인증번호':'EXHST_GAS_CERT_NO', 
#     '검토구분':'RVW_SE', 
#     '대표제작사명':'RPRS_MNFCTR_NM', 
#     '대표차명':'RPRS_VHCNM', 
#     '대표차종':'RPRS_VHCTY_CD', 
#     '대표차연료':'RPRS_FUEL', 
#     '최초연식':'FRST_YRIDNW', 
#     '열화도':'SI', 
#     '테이블생성일자':'LOAD_DT', 
#     # '검사방법':'INSP_MTHD', 
# }
cdict = {
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '검사방법':'INSP_MTHD', 
    '검토구분':'RVW_SE', 
    '제작사명':'RPRS_MNFCTR_NM', 
    '차명':'RPRS_VHCNM', 
    '차종':'RPRS_VHCTY_CD', 
    '연료':'RPRS_FUEL', 
    '최초연식':'FRST_YRIDNW', 
    '열화도':'SI', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_SI = STD_BD_DAT_GRD4_SI.rename(columns=cdict)
STD_BD_DAT_GRD4_SI.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RVW_SE', 'RPRS_MNFCTR_NM',
       'RPRS_VHCNM', 'RPRS_VHCTY_CD', 'RPRS_FUEL', 'FRST_YRIDNW', 'SI',
       'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_GRD4_SI

In [446]:
# expdf = STD_BD_DAT_GRD4_SI
# table_nm = 'STD_BD_DAT_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [447]:
# # 0s
# STD_BD_DAT_GRD4_SI.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_SI.csv'), index=False)

# code end